In [28]:
import numpy as np
import gym
import random

Creating environmnent:
- 5x5 grid world
- Taxi is spawned randomly
- Passenger is spawned randomly in one of R,B,G,Y and wants to be dropped off in R,B,G,Y

Reward system:

- -1 for each timestep
- +20 for successfully deliver
- -10 for bad actions (pickup or putdown the passenger outside their destination)

In [29]:
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Q-table creation and init

In [30]:
#OpenAI Gym methods to get action and observational space
state_space = env.observation_space.n
action_space = env.action_space.n
print("There are ", state_space, " possible states")
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [37]:
Q = np.zeros((state_space, action_space))
print(Q.shape)

(500, 6)


Define hyperparameters

In [32]:
total_episodes = 25000 #training iter
total_test_episodes = 10 #test iter
max_steps = 200 #max steps per episode (no. steps to fail)

learning_rate = 0.01
gamma = 0.99 #discount rate to prefer shorter sequences

# Exploration params
epsilon = 1.0 #exploration rate
max_epsilon = 1.0 #at start
min_epsilon = 0.001 #at the end / min 
decay_rate = 0.01 #exponential decay rate for epsilon

With prob 1-epsilon, agent only does exploitation (tries action with max state-action pair value)

With prob epsilon, agent only does exploration (tries random action)

In [33]:
def epsilon_greedy_policy(Q, state):
    # if rand > epsilon -> exploitation
    if (random.uniform(0,1) > epsilon):
        action = np.argmax(Q[state])
    # else -> exploration
    else:
        action = env.action_space.sample()
    
    return action

Define Q-learning algo and train:

In [40]:
for episode in range(total_episodes):
    # Reset
    state = env.reset()
    step = 0
    done = False
    
    # Reduce epsilon (to lower exploration per iter)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate * episode)
    
    for step in range(max_steps):
        action = epsilon_greedy_policy(Q, state)
        
        # Take action (a) and observe the outcome state and reward
        new_state, reward, done, info = env.step(action)
        
        # Update Q
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma * np.max(Q[new_state]) - Q[state][action])
        
        if done == True:
            break
        
        # New state is current state
        state = new_state

Illustration of the agent play

In [48]:
import time

rewards = []
frames = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("*" * 10)
    print("Test Episode ", episode)
    for step in range(max_steps):
        env.render()
        # Take action that has max expected future reward given state (plain greedy algo)
        action = np.argmax(Q[state][:])
        new_state, reward, done, info = env.step(action)
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            break
            
        state = new_state

env.close()
print("Score over time: ", str(sum(rewards)/total_test_episodes))

**********
Test Episode  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| :_: : 